In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [2]:
csv_file = "Resources/Univ_Info.csv"
univ_info_df = pd.read_csv(csv_file)
univ_info_df.head()

,UnitID,Institution_Name,Institution_size_category,Institution_entity_name,Institution_name_alias,City_location,Address,State,Zip
0,240985,\tEducational Technical College-Recinto de Bay...,1,Educational Technical College-Recinto de Bayamon,NaN,Bayamon,1685 CARR #2 KL 11.2,PR,959
1,177834,A T Still University of Health Sciences,2,A T Still University of Health Sciences,NaN,Kirksville,800 W Jefferson,MO,63501
2,180203,Aaniiih Nakoda College,1,Aaniiih Nakoda College,NaN,Harlem,269 Blackfeet Avenue Agency,MT,59526
3,459523,ABC Beauty Academy,1,ABC Beauty Academy,NaN,Richardson,2109 Buckingham Rd.,TX,75081
4,485500,ABCO Technology,1,ABCO Technology,NaN,Inglewood,11222 S. La Cienega Blvd Ste 588,CA,90304-1551


In [3]:
csv_file = "Resources/Labels.csv"
labels_df = pd.read_csv(csv_file)
labels_df.head()

,Value,ValueLabel
0,1,"Under 1,000"
1,2,"1,000 - 4,999"
2,3,"5,000 - 9,999"
3,4,"10,000 - 19,999"
4,5,"20,000 and above"


In [4]:
csv_file = "Resources/Univ_Admission_Data_2017.csv"
univ_admission_data_2017_df = pd.read_csv(csv_file)
univ_admission_data_2017_df.head()

,UnitID,Institution Name,Grand total,Total men,Total women
0,240985,\tEducational Technical College-Recinto de Bay...,383.0,133.0,250.0
1,177834,A T Still University of Health Sciences,NaN,NaN,NaN
2,180203,Aaniiih Nakoda College,122.0,52.0,70.0
3,459523,ABC Beauty Academy,NaN,NaN,NaN
4,485500,ABCO Technology,50.0,45.0,5.0


### Clean DataFrame

In [5]:
labels_df = labels_df.rename(columns={"Value": "value_id","ValueLabel":"value_label" })
labels_df

,value_id,value_label
0,1,"Under 1,000"
1,2,"1,000 - 4,999"
2,3,"5,000 - 9,999"
3,4,"10,000 - 19,999"
4,5,"20,000 and above"


In [6]:
univ_info_df = univ_info_df.rename(columns={"UnitID":"unit_id",
                                            "Institution_Name":"institution_name",
                                            "Institution_size_category":"institution_size_category",
                                            "Institution_entity_name": "institution_entity_name",
                                            "Institution_name_alias": "institution_name_alias",
                                            "City_location":"city_location",
                                            "Address":"address",
                                            "State":"state",
                                            "Zip" : "zip"
                                           }
                                  )
univ_info_df.head()

,unit_id,institution_name,institution_size_category,institution_entity_name,institution_name_alias,city_location,address,state,zip
0,240985,\tEducational Technical College-Recinto de Bay...,1,Educational Technical College-Recinto de Bayamon,NaN,Bayamon,1685 CARR #2 KL 11.2,PR,959
1,177834,A T Still University of Health Sciences,2,A T Still University of Health Sciences,NaN,Kirksville,800 W Jefferson,MO,63501
2,180203,Aaniiih Nakoda College,1,Aaniiih Nakoda College,NaN,Harlem,269 Blackfeet Avenue Agency,MT,59526
3,459523,ABC Beauty Academy,1,ABC Beauty Academy,NaN,Richardson,2109 Buckingham Rd.,TX,75081
4,485500,ABCO Technology,1,ABCO Technology,NaN,Inglewood,11222 S. La Cienega Blvd Ste 588,CA,90304-1551


In [7]:
univ_admission_data_2017_df = univ_admission_data_2017_df.rename(columns={"UnitID":"unit_id",
                                                                          "Institution Name": "institution_name",
                                                                          "Grand total":"total_2017",
                                                                          "Total men":"total_men_2017",
                                                                          "Total women":"total_women_2017"
                                                                           }
                                                                  )
univ_admission_data_2017_df.head()

,unit_id,institution_name,total_2017,total_men_2017,total_women_2017
0,240985,\tEducational Technical College-Recinto de Bay...,383.0,133.0,250.0
1,177834,A T Still University of Health Sciences,NaN,NaN,NaN
2,180203,Aaniiih Nakoda College,122.0,52.0,70.0
3,459523,ABC Beauty Academy,NaN,NaN,NaN
4,485500,ABCO Technology,50.0,45.0,5.0


### Connect to local database

In [8]:
rds_connection_string = "postgres:postgres@localhost:5432/ETL_Univ_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [9]:
engine.table_names()

['labels', 'univ_info', 'univ_admission_data_2017']

### Use pandas to load csv converted DataFrame into database

In [11]:
labels_df.to_sql(name='labels', con=engine, if_exists='append', index=False)

In [12]:
univ_info_df.to_sql(name='univ_info', con=engine, if_exists='append', index=False)

In [13]:
univ_admission_data_2017_df.to_sql(name='univ_admission_data_2017', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the table

In [14]:
pd.read_sql_query('select * from labels', con=engine).head()

,value_id,value_label
0,1,"Under 1,000"
1,2,"1,000 - 4,999"
2,3,"5,000 - 9,999"
3,4,"10,000 - 19,999"
4,5,"20,000 and above"


In [15]:
pd.read_sql_query('select * from univ_info', con=engine).head()

,unit_id,institution_name,institution_size_category,institution_entity_name,institution_name_alias,city_location,address,state,zip
0,240985,\tEducational Technical College-Recinto de Bay...,1,Educational Technical College-Recinto de Bayamon,None,Bayamon,1685 CARR #2 KL 11.2,PR,959
1,177834,A T Still University of Health Sciences,2,A T Still University of Health Sciences,None,Kirksville,800 W Jefferson,MO,63501
2,180203,Aaniiih Nakoda College,1,Aaniiih Nakoda College,None,Harlem,269 Blackfeet Avenue Agency,MT,59526
3,459523,ABC Beauty Academy,1,ABC Beauty Academy,None,Richardson,2109 Buckingham Rd.,TX,75081
4,485500,ABCO Technology,1,ABCO Technology,None,Inglewood,11222 S. La Cienega Blvd Ste 588,CA,90304-1551


In [16]:
pd.read_sql_query('select * from univ_admission_data_2017', con=engine).head()

,unit_id,institution_name,total_2017,total_men_2017,total_women_2017
0,240985,\tEducational Technical College-Recinto de Bay...,383.0,133.0,250.0
1,177834,A T Still University of Health Sciences,NaN,NaN,NaN
2,180203,Aaniiih Nakoda College,122.0,52.0,70.0
3,459523,ABC Beauty Academy,NaN,NaN,NaN
4,485500,ABCO Technology,50.0,45.0,5.0


# Joins

In [17]:
joined_df_1 = pd.merge(univ_admission_data_2017_df, univ_info_df,on='unit_id')
joined_df_1.head()

,unit_id,institution_name_x,total_2017,total_men_2017,total_women_2017,institution_name_y,institution_size_category,institution_entity_name,institution_name_alias,city_location,address,state,zip
0,240985,\tEducational Technical College-Recinto de Bay...,383.0,133.0,250.0,\tEducational Technical College-Recinto de Bay...,1,Educational Technical College-Recinto de Bayamon,NaN,Bayamon,1685 CARR #2 KL 11.2,PR,959
1,177834,A T Still University of Health Sciences,NaN,NaN,NaN,A T Still University of Health Sciences,2,A T Still University of Health Sciences,NaN,Kirksville,800 W Jefferson,MO,63501
2,180203,Aaniiih Nakoda College,122.0,52.0,70.0,Aaniiih Nakoda College,1,Aaniiih Nakoda College,NaN,Harlem,269 Blackfeet Avenue Agency,MT,59526
3,459523,ABC Beauty Academy,NaN,NaN,NaN,ABC Beauty Academy,1,ABC Beauty Academy,NaN,Richardson,2109 Buckingham Rd.,TX,75081
4,485500,ABCO Technology,50.0,45.0,5.0,ABCO Technology,1,ABCO Technology,NaN,Inglewood,11222 S. La Cienega Blvd Ste 588,CA,90304-1551


In [18]:
joined_df_2 = pd.merge(joined_df_1,labels_df, left_on='institution_size_category', right_on='value_id')
joined_df_2.head()

,unit_id,institution_name_x,total_2017,total_men_2017,total_women_2017,institution_name_y,institution_size_category,institution_entity_name,institution_name_alias,city_location,address,state,zip,value_id,value_label
0,240985,\tEducational Technical College-Recinto de Bay...,383.0,133.0,250.0,\tEducational Technical College-Recinto de Bay...,1,Educational Technical College-Recinto de Bayamon,NaN,Bayamon,1685 CARR #2 KL 11.2,PR,959,1,"Under 1,000"
1,180203,Aaniiih Nakoda College,122.0,52.0,70.0,Aaniiih Nakoda College,1,Aaniiih Nakoda College,NaN,Harlem,269 Blackfeet Avenue Agency,MT,59526,1,"Under 1,000"
2,459523,ABC Beauty Academy,NaN,NaN,NaN,ABC Beauty Academy,1,ABC Beauty Academy,NaN,Richardson,2109 Buckingham Rd.,TX,75081,1,"Under 1,000"
3,485500,ABCO Technology,50.0,45.0,5.0,ABCO Technology,1,ABCO Technology,NaN,Inglewood,11222 S. La Cienega Blvd Ste 588,CA,90304-1551,1,"Under 1,000"
4,461892,Abcott Institute,151.0,10.0,141.0,Abcott Institute,1,Abcott Institute,NaN,Southfield,16250 Northland Drive Suite 205,MI,48075-5227,1,"Under 1,000"


In [20]:
df_grp = joined_df_2.groupby(['value_id']).mean()
df_grp

,unit_id,total_2017,total_men_2017,total_women_2017,institution_size_category
value_id,,,,,
1,337971.303037,224.246009,79.784873,144.461136,1.0
2,207170.854331,2185.100472,912.447067,1272.653405,2.0
3,193189.492032,6279.824701,2677.165339,3602.659363,3.0
4,190680.550432,12086.731214,5381.505780,6705.225434,4.0
5,181047.099548,26844.669683,12337.149321,14507.520362,5.0


In [22]:
df_grp['percentage_women'] = 100*df_grp['total_women_2017'] / df_grp['total_2017']
df_grp.head()

,unit_id,total_2017,total_men_2017,total_women_2017,institution_size_category,percentage_women
value_id,,,,,,
1,337971.303037,224.246009,79.784873,144.461136,1.0,64.420828
2,207170.854331,2185.100472,912.447067,1272.653405,2.0,58.242329
3,193189.492032,6279.824701,2677.165339,3602.659363,3.0,57.368789
4,190680.550432,12086.731214,5381.505780,6705.225434,4.0,55.475921
5,181047.099548,26844.669683,12337.149321,14507.520362,5.0,54.042462
